In [59]:
import pandas as pd

Remember to fill pv forecast!

In [60]:
def read_data():
    data_ = pd.DataFrame()
    folder_path = 'data'
    for file_name in ['features_2018.csv', 'features_2019.csv', 'features_2020.csv']:
        file_path = '/'.join([folder_path, file_name])
        data_ = pd.concat([
            data_,
            pd.read_csv(file_path, index_col=0)
        ])
    data_.index = pd.to_datetime(data_.index)

    prices = pd.read_csv('data/prices.csv',
                         parse_dates=['delivery_start_utc'],
                         )
    prices['delivery_start_utc'] = prices['delivery_start_utc'].dt.tz_localize('UTC')
    prices['delivery_start_utc'] = prices['delivery_start_utc'].dt.tz_convert('Europe/Warsaw')
    prices['delivery_start_utc'] = prices['delivery_start_utc'].dt.tz_localize(None)
    prices.set_index('delivery_start_utc', inplace=True)
    prices = prices[['price']]

    return pd.merge(prices, data_, left_index=True, right_index=True)

data = read_data()

# Fill missings

In [76]:
def fill_pv_generation_missings(data):
    pv_profile = pd.read_pickle('data/pv_profile.pkl')
    pv_profile.set_index('date', inplace=True)
    pv_profile = pv_profile[['system_generation']]
    pv_profile.index = pd.to_datetime(pv_profile.index)
    missing_solar_timestamps = data[data['generation_solar_actual'].isna()].index
    solar_generation = pv_profile[pv_profile.index.isin(missing_solar_timestamps)]
    data.loc[data.index.isin(solar_generation.index), 'generation_solar_actual'] = solar_generation['system_generation']
    data.loc[data.index < pv_profile.index.min(), 'generation_solar_actual'] = 0
    # Last 5 missings in November after 6 p.m.
    data.loc[data['generation_solar_actual'].isna(), 'generation_solar_actual'] = 0
    return data

def fill_all_missings(data):
    missings = data.isnull().sum()
    missing_columns = missings[missings>0].index
    data[missing_columns] = data[missing_columns].fillna(data[missing_columns].median())
    return data

data_filled_with_pv = fill_pv_generation_missings(data=data)
data_filled = fill_all_missings(data=data_filled_with_pv)

# Filter columns

In [77]:
data_filtered = data_filled.drop(columns=[
    'capacity_njwcd_forecast',
    'capacity_jwcd_forecast',
    'reserve1_forecast',
    'load_forecast',
    'reserve2_forecast',
    'generation_jwcd_forecast',
    'generation_pi_actual',
    'generation_irz_actual',
    'co2_price_eur_actual',
    'UA_EXP',
])

# Get lags and filter

In [99]:
def get_lags(df, columns, lags):
    df_copy = df.copy()
    for column in columns:
        for lag in lags:
            if column not in df.columns:
                print(f'Column {column} not found in given data frame columns')
                return df
            df_copy[f'{column}_lag_{lag}'] = df_copy[column].shift(lag)
    return df_copy

fixed_columns =  ['forecast_PLN', 'weekday', 'month'] #Not to be lagged
columns_to_lag = [column for column in data_filtered.columns if column not in fixed_columns]
lags = [24, 7*24]
data_lagged = get_lags(df=data_filtered, columns=columns_to_lag, lags=lags)
data_lagged = data_lagged[max(lags):]

In [105]:
columns_to_drop = [
    'load_actual',
    'generation_jwcd_actual',
    'generation_njwcd_actual',
    'crossborder_sync_balance_actual',
    'crossborder_async_balance_actual',
    'CEPS_EXP',
    'CEPS_IMP',
    'SEPS_EXP',
    'SEPS_IMP',
    '50HzT_EXP',
    '50HzT_IMP',
    'SVK_EXP',
    'SVK_IMP',
    'UA_IMP',
    'LIT_EXP',
    'LIT_IMP',
    'generation_wind_actual',
    'generation_solar_actual',
    'Biomass 200 generation',
    'Gas 200 generation',
    'Gas 500 generation',
    'HPS 100 generation',
    'HPS 200 generation',
    'Hard coal 100 generation',
    'Hard coal 1000 generation',
    'Hard coal 200 generation',
    'Hard coal 300 generation',
    'Hard coal 500 generation',
    'Lignite 1000 generation',
    'Lignite 200 generation',
    'Lignite 300 generation',
    'Lignite 500 generation',
    'co2_price_pln_actual_lag_168',
    'generation_wind_forecast_lag_24',
    'generation_wind_forecast_lag_168',
    'generation_solar_forecast_lag_24',
    'generation_solar_forecast_lag_168',
    'generation_wind_actual_lag_168',
    'generation_solar_actual_lag_168',
    'Gas 500 available_lag_24',
    'Gas 500 available_lag_168',
    'Hard coal 100 available_lag_24',
    'Hard coal 100 available_lag_168',
    'Hard coal 1000 available_lag_24',
    'Hard coal 1000 available_lag_168',
    'Hard coal 200 available_lag_24',
    'Hard coal 200 available_lag_168',
    'Hard coal 500 available_lag_24',
    'Hard coal 500 available_lag_168',
    'Lignite 200 available_lag_24',
    'Lignite 200 available_lag_168',
    'Lignite 300 available_lag_24',
    'Lignite 300 available_lag_168',
    'Hard coal 300 available_lag_24',
    'Hard coal 300 available_lag_168',
    'Lignite 500 available_lag_24',
    'Lignite 500 available_lag_168',
    'Lignite 1000 available_lag_24',
    'Lignite 1000 available_lag_168',
    'Biomass 200 available_lag_24',
    'Biomass 200 available_lag_168',
    'Gas 200 available_lag_24',
    'Gas 200 available_lag_168',
    'HPS 100 available_lag_24',
    'HPS 100 available_lag_168',
    'HPS 200 available_lag_24',
    'HPS 200 available_lag_168',
]
data_lagged_filtered = data_lagged.drop(columns=columns_to_drop)

In [106]:
final = data_lagged_filtered.reset_index().rename(columns={'index': 'timestamp'})

In [107]:
final.to_csv('data/full_data_filtered.csv', index=False)